In [6]:
from pathlib import Path
from datetime import datetime
import yaml
import json
import logging
from rich.logging import RichHandler
from rich.console import Console
from typing import List, Dict
# Create a new project directory and move README.md there
project_dir = Path(r"C:\Users\devil\Documents\Projects\Fetch")
project_dir.mkdir(exist_ok=True)



In [6]:
# conda env create -f environment.yml
# conda activate pickleball_llm
! pip install uv
# ! uv --version



In [10]:
# uv pip install -r requirements.txt
# # or
# ! uv pip install .
! uv pip install "C:\Users\devil\Documents\Projects\Fetch"


Using Python 3.11.11 environment at: C:\Users\devil\.conda\envs\test
Resolved 88 packages in 6.05s
   Building ml-apprentice-project @ file:///C:/Users/devil/Documents/Projects/Fetch
 Downloaded pywin32
 Downloaded mlflow-skinny
  × Failed to build `ml-apprentice-project @
  │ file:///C:/Users/devil/Documents/Projects/Fetch`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit code: 1)

      [stderr]
      Traceback (most recent call last):
        File "<string>", line 11, in <module>
        File
      "C:\Users\devil\AppData\Local\uv\cache\builds-v0\.tmpZnR2yw\Lib\site-packages\setuptools\build_meta.py",
      line 435, in build_wheel
          return _build(['bdist_wheel'])
                 ^^^^^^^^^^^^^^^^^^^^^^^
        File
      "C:\Users\devil\AppData\Local\uv\cache\builds-v0\.tmpZnR2yw\Lib\site-packages\setuptools\build_meta.py",
      line 426, in _build
          return self._build_with_temp_dir(
                 ^^^^^^^^

In [9]:
! uv pip freeze > requirements.lock.txt


Using Python 3.11.11 environment at: C:\Users\devil\.conda\envs\test


In [10]:
# ! uv pip install -r requirements.txt
# 

error: File not found: `requirements.txt`


In [ ]:
! pip install rich


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\devil\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/0d/9b/63f4c7ebc259242c89b3acafdb37b41d1185c07ff0011164674e9076b491/rich-14.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for markdown-it-py>=2.2.0 from https://files.pythonhosted.org/packages/42/d7/1ec15b46af6af88f19b8e5ffea08fa375d433c998b8a7639e76935c14f1f/markdown_it_py-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for mdurl~=0.1 from https://files.pythonhosted.org/packages/b3/38/89ba8ad64ae25be8de66a6d463314cf1eb366222074cfda9ee839c56a4b4/mdurl-0.1.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/243.2 kB ? eta -:--:--
   ------------------------- -------------- 153.6/243.2 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 243.2/243.2 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
   ---------------------------------------- 87.5/87.5 kB ? eta 0:00:00


In [4]:
DEFAULT_YAML_FILE = "project_structure.yaml"
DEFAULT_RESULT_OUTPUT = "creation_result.json"

In [7]:

# Setup rich logger
console = Console()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)]
)
logger = logging.getLogger("project_builder")


def load_yaml(yaml_path: str) -> Dict:
    """Load a YAML file and return its content as a dictionary."""
    try:
        with open(yaml_path, "r") as file:
            return yaml.safe_load(file)
    except Exception as e:
        logger.exception("Failed to load YAML file.")
        raise


def create_file(path: Path, created: List[str], skipped: List[str], errors: List[Dict[str, str]]) -> None:
    """Create a file if it doesn't exist."""
    try:
        if path.exists():
            skipped.append(str(path))
        else:
            path.parent.mkdir(parents=True, exist_ok=True)
            path.touch()
            created.append(str(path))
    except Exception as e:
        errors.append({"file": str(path), "error": str(e)})


def process_structure(base_path: Path, structure, created: List[str], skipped: List[str], errors: List[Dict[str, str]]) -> None:
    """Recursively process the structure and create files."""
    if isinstance(structure, list):
        for item in structure:
            if isinstance(item, str):
                create_file(base_path / item, created, skipped, errors)
            elif isinstance(item, dict):
                for k, v in item.items():
                    process_structure(base_path / k, v, created, skipped, errors)
    elif isinstance(structure, dict):
        for k, v in structure.items():
            process_structure(base_path / k, v, created, skipped, errors)


def save_json(data: Dict, output_path: str) -> None:
    """Save a dictionary to a JSON file."""
    try:
        Path(output_path).write_text(json.dumps(data, indent=2))
        logger.info(f"[green]Summary saved to {output_path}[/green]")
    except Exception as e:
        logger.error(f"[red]Failed to write summary: {e}[/red]")


def build_project_from_yaml(yaml_file: str = DEFAULT_YAML_FILE, result_output: str = DEFAULT_RESULT_OUTPUT) -> None:
    """Build the project directory and file structure from YAML."""
    try:
        yaml_content = load_yaml(yaml_file)
        structure = yaml_content.get("project", {}).get("structure", {})
    except Exception:
        logger.error("[bold red]YAML parsing failed. Exiting.[/bold red]")
        return

    created, skipped, errors = [], [], []
    process_structure(Path("."), structure, created, skipped, errors)

    result = {
        "timestamp": datetime.now().isoformat(),
        "created": created,
        "skipped": skipped,
        "errors": errors,
    }

    save_json(result, result_output)


# Entry point
if __name__ == "__main__":
    build_project_from_yaml()


[18:10:55] INFO     [green]Summary saved to creation_result.json[/green]                           ]8;id=810368;file://C:\Users\devil\AppData\Local\Temp\ipykernel_9980\4209271769.py\4209271769.py]8;;\:]8;id=663052;file://C:\Users\devil\AppData\Local\Temp\ipykernel_9980\4209271769.py#53\53]8;;\